### Bewegte Kamera im Stereotest

In [ ]:
import numpy as np
import cv2 as cv
import copy
import time
bs = 15

cap = cv.VideoCapture(0)
ret, old = cap.read()

while True:
    stereo = cv.StereoBM_create(numDisparities=0, blockSize=bs)
    ret, new = cap.read()
    imgR =cv.cvtColor(new, cv.COLOR_BGR2GRAY)
    imgL =cv.cvtColor(old, cv.COLOR_BGR2GRAY)

    disparity = stereo.compute(imgL, imgR)
    stereo = cv.normalize(disparity, None, alpha = 0, beta = 1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
    
    cv.imshow("R",imgR)
    cv.imshow("L",imgL)
    cv.imshow("stereo", stereo)
    
    key = cv.waitKey(1)
    if key == ord('q'):
        break
    if key == ord('x'):
        bs+=2
    if key == ord('y'):
        bs-=2
    if key == ord('p'):
        print(bs)
    old = new
    time.sleep(0.2)
        
cv.destroyAllWindows()
cap.release()

error: OpenCV(4.5.4-dev) D:/a/opencv-python/opencv-python/opencv/modules/highgui/src/precomp.hpp:155: error: (-215:Assertion failed) src_depth != CV_16F && src_depth != CV_32S in function 'convertToShow'


### Stereo auf Turm

In [1]:
import numpy as np
import cv2 as cv
import copy
import time
bs = 5

capl = cv.VideoCapture("Stereo/Stereo_Turm (1).mp4")
capr = cv.VideoCapture("Stereo/Stereo_Turm (2).mp4")

while True:
    stereo = cv.StereoBM_create(numDisparities=0, blockSize=bs)
    ret, right = capr.read()
    ret, left = capl.read()
    imgR = cv.cvtColor(right, cv.COLOR_BGR2GRAY)
    imgL = cv.cvtColor(left, cv.COLOR_BGR2GRAY)

    disparity = stereo.compute(imgL, imgR)
    stereo = cv.normalize(disparity, None, alpha = 0, beta = 1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
    
    cv.imshow("R",imgR)
    cv.imshow("L",imgL)
    cv.imshow("stereo", stereo)
    
    key = cv.waitKey(1)
    if key == ord('q'):
        break
    if key == ord('x'):
        bs+=2
    if key == ord('y'):
        bs-=2
    if key == ord('p'):
        print(bs)
        
cv.destroyAllWindows()
capl.release()
capr.release()

error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\stereobm.cpp:1190: error: (-211:One of the arguments' values is out of range) SADWindowSize must be odd, be within 5..255 and be not larger than image width or height in function 'cv::StereoBMImpl::compute'


### Seitenansicht 1 Vid

In [28]:
import imutils
import cv2
import numpy as np

vs = cv2.VideoCapture("test2.mp4")
ret, frame = vs.read()
old = None
count = 1

max_cnt_area = 700
upper_border_height = 40
lower_border_height = 500
shredder_left = 430
shredder_right = 830
margin = 80

dangerzone_left = ((margin, upper_border_height), (shredder_left,lower_border_height))
dangerzone_right = ((shredder_right, upper_border_height), (frame.shape[1]-margin, lower_border_height))
shredderzone = ((shredder_left, upper_border_height), (shredder_right,lower_border_height))

def overlap(mid):
    if dangerzone_right[0][0] < mid[0] < dangerzone_right[1][0] and dangerzone_right[0][1] < mid[1] < dangerzone_right[1][1]:
        return "Rechts"
    elif dangerzone_left[0][0] < mid[0] < dangerzone_left[1][0] and dangerzone_left[0][1] < mid[1] < dangerzone_left[1][1]:
        return "Links"
    else:
        return None

#while True: #Endlosschleife für Live-Video, hier forschleife über endliches Video
for i in range(1,int(vs.get(cv2.CAP_PROP_FRAME_COUNT))) :
    ret, frame = vs.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
        
    if old is None:
        old = gray
        continue
        
    frameDelta = cv2.absdiff(old, gray)
    thresh = cv2.threshold(frameDelta, 5, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    for c in cnts:
        # if the contour is too big, ignore it
        if cv2.contourArea(c) > max_cnt_area:
            continue
        (x, y, w, h) = cv2.boundingRect(c)
        tl = (x, y)
        br = (x+w, y+h)
        mid = (x+w//2,y+h//2)
        ov = overlap(mid)
        if ov:
            cv2.putText(frame,ov,(20,700),cv2.FONT_HERSHEY_COMPLEX,1,(0,163,255),2)
        cv2.circle(frame, mid, 2, (0, 0, 255), 2)
        cv2.rectangle(frame, tl, br, (0, 255, 0), 2)
    
    if count%2 == 0:
        old = gray #Setze old auf aktuelles Bild, um es im nächsten Durchlauf zu vergleichen
        count = 1
    count+=1
    
    #Label
    cv2.rectangle(frame, dangerzone_left[0], dangerzone_left[1], (0, 255, 255), 1) #dangerzone left
    cv2.rectangle(frame, dangerzone_right[0], dangerzone_right[1], (0, 255, 255), 1) #dangerzone right
    cv2.rectangle(frame, shredderzone[0], shredderzone[1], (0, 0, 255), 1) #shredderzone
    
    #Bilder anzeigen
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame", frame)
    cv2.imshow("Frame Delta", frameDelta)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break
cv2.destroyAllWindows()
vs.release()

### Seitenansicht alle Vids

In [3]:
import imutils
import cv2
import numpy as np

for i in range(1,10):
    vs = cv2.VideoCapture("Seitentest/Seite ("+str(i)+").mp4")    
    ret, frame = vs.read()
    old = None
    oldpos = 0

    max_cnt_area = 800
    upper_border_height = 40
    lower_border_height = 500
    shredder_left = 430
    shredder_right = 830
    margin = 80

    dangerzone_left = ((margin, upper_border_height), (shredder_left,lower_border_height))
    dangerzone_right = ((shredder_right, upper_border_height), (frame.shape[1]-margin, lower_border_height))
    shredderzone = ((shredder_left, upper_border_height), (shredder_right,lower_border_height))

    def overlap(mid):
        if dangerzone_right[0][0] < mid[0] < dangerzone_right[1][0] and dangerzone_right[0][1] < mid[1] < dangerzone_right[1][1]:
            return "Rechts"
        elif dangerzone_left[0][0] < mid[0] < dangerzone_left[1][0] and dangerzone_left[0][1] < mid[1] < dangerzone_left[1][1]:
            return "Links"
        else:
            return None

    #while True: #Endlosschleife für Live-Video, hier forschleife über endliches Video
    for i in range(1,int(vs.get(cv2.CAP_PROP_FRAME_COUNT))) :
        ret, frame = vs.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)

        if old is None:
            old = gray
            continue

        frameDelta = cv2.absdiff(old, gray)
        thresh = cv2.threshold(frameDelta, 5, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=1)
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        for c in cnts:
            # if the contour is too big, ignore it
            if cv2.contourArea(c) > max_cnt_area:
                continue
            (x, y, w, h) = cv2.boundingRect(c)
            tl = (x, y)
            br = (x+w, y+h)
            mid = (x+w//2,y+h//2)
            ov = overlap(mid)
            if ov:
                cv2.putText(frame,ov,(20,700),cv2.FONT_HERSHEY_COMPLEX,1,(0,163,255),2)
            cv2.circle(frame, mid, 2, (0, 0, 255), 2)
            cv2.rectangle(frame, tl, br, (0, 255, 0), 2)

        old = gray #Setze old auf aktuelles Bild, um es im nächsten Durchlauf zu vergleichen

        #Label
        cv2.rectangle(frame, dangerzone_left[0], dangerzone_left[1], (0, 255, 255), 1) #dangerzone left
        cv2.rectangle(frame, dangerzone_right[0], dangerzone_right[1], (0, 255, 255), 1) #dangerzone right
        cv2.rectangle(frame, shredderzone[0], shredderzone[1], (0, 0, 255), 1) #shredderzone

        #Bilder anzeigen
        cv2.imshow("Thresh", thresh)
        cv2.imshow("Frame", frame)
        cv2.imshow("Frame Delta", frameDelta)
        key = cv2.waitKey(1)
        if key == ord("q"):
            break
    cv2.destroyAllWindows()
    vs.release()

### Live-Kamera-Test

In [2]:
import imutils
import cv2
import numpy as np

vs = cv2.VideoCapture(0)
vs.set(3, 1920)
vs.set(4, 1080)
ret, frame = vs.read()
old = None

max_cnt_area = 700
upper_border_height = 40
lower_border_height = 700
shredder_left = 700
shredder_right = 1200
margin = 80

dangerzone_left = ((margin, upper_border_height), (shredder_left,lower_border_height))
dangerzone_right = ((shredder_right, upper_border_height), (frame.shape[1]-margin, lower_border_height))
shredderzone = ((shredder_left, upper_border_height), (shredder_right,lower_border_height))

def overlap(mid):
    if dangerzone_right[0][0] < mid[0] < dangerzone_right[1][0] and dangerzone_right[0][1] < mid[1] < dangerzone_right[1][1]:
        return "Rechts"
    elif dangerzone_left[0][0] < mid[0] < dangerzone_left[1][0] and dangerzone_left[0][1] < mid[1] < dangerzone_left[1][1]:
        return "Links"
    else:
        return None

while True: #Endlosschleife für Live-Video, hier forschleife über endliches Video
    ret, frame = vs.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
        
    if old is None:
        old = gray
        continue
        
    frameDelta = cv2.absdiff(old, gray)
    thresh = cv2.threshold(frameDelta, 5, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    for c in cnts:
        # if the contour is too big, ignore it
        if cv2.contourArea(c) > max_cnt_area:
            continue
        (x, y, w, h) = cv2.boundingRect(c)
        tl = (x, y)
        br = (x+w, y+h)
        mid = (x+w//2,y+h//2)
        ov = overlap(mid)
        if ov:
            cv2.putText(frame,ov,(20,700),cv2.FONT_HERSHEY_COMPLEX,1,(0,163,255),2)
        cv2.circle(frame, mid, 2, (0, 0, 255), 2)
        cv2.rectangle(frame, tl, br, (0, 255, 0), 2)
    
    old = gray #Setze old auf aktuelles Bild, um es im nächsten Durchlauf zu vergleichen
    
    #Label
    cv2.rectangle(frame, dangerzone_left[0], dangerzone_left[1], (0, 255, 255), 1) #dangerzone left
    cv2.rectangle(frame, dangerzone_right[0], dangerzone_right[1], (0, 255, 255), 1) #dangerzone right
    cv2.rectangle(frame, shredderzone[0], shredderzone[1], (0, 0, 255), 1) #shredderzone
    
    #Bilder anzeigen
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame", frame)
    cv2.imshow("Frame Delta", frameDelta)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break
cv2.destroyAllWindows()
vs.release()

### Rotortracking

1. Bild wird mit vorherigem Bild verglichen -> Unterschiede = Bewegung. Differenzbild wird durch threshhold weiterverarbeitet.
2. Konturen werden ermittelt.
3. Für jede Kontur wird der Punkt mit der größten euklidischen DIstanz zum Mittelpunkt ermittelt.
4. Durch Imaginäre Zahl wird für jeden dieser Punkte der um 120° gedrehte Punkt gesucht.
5. Zu diesem Punkt wird eine dickere Linie erstellt.
6. Wenn diese Linie zu viele Pixel der Konturen-Layer überschneidet, wird angenommen, dass es sich um eine Windradkontur handelt.

(Punkt 5 sollte überarbeitet werden, da immer mit Mittelpunkt überschnitten wird) 

In [143]:
import imutils
import cv2
import numpy as np
import time

vs = cv2.VideoCapture("Frontansicht.mp4")
vs.set(3, 1920)
vs.set(4, 1080)
ret, frame = vs.read()
old = None
center = np.array((632,203))
line_image = np.zeros(frame.shape)

for i in range(1,int(vs.get(cv2.CAP_PROP_FRAME_COUNT))):
    ret, frame = vs.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
        
    if old is None:
        old = gray
        continue
        
    frameDelta = cv2.absdiff(old, gray)
    thresh = cv2.threshold(frameDelta, 5, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    for c in cnts:
        line_image = np.zeros(frame.shape[:-1])
        
        #Koordinaten des am weitesten von center entfernten Punkts einer Kontur
        distances = [np.linalg.norm(center-i[0]) for i in c]
        point = c[np.argsort(distances)[-1]][0]
        
        #Angewinkelte Linie berechnen
        z = (complex(point[0],point[1]) - complex(center[0],center[1])) * (-0.4999999999999998-0.8660254037844387j) + complex(center[0],center[1])
        angled_point_coords = np.int32([z.real,z.imag])
        
        cv2.line(line_image, angled_point_coords, center, (255,255,255), 15)
        cv2.line(frame, angled_point_coords, center, (255,255,255), 15)
        
        if np.sum(np.int32(line_image) & thresh) > 50000:
            cv2.line(frame, point, center, (0,0,255), 2)
        else:
            cv2.line(frame, point, center, (255,0,0), 2)

    cv2.circle(frame, center, 2, (255, 0, 255), 2)
    old = gray #Setze old auf aktuelles Bild, um es im nächsten Durchlauf zu vergleichen

    #Bilder anzeigen
    cv2.imshow("Frame", frame)
    #cv2.imshow("line", line_image)
    #cv2.imshow("Thresh", thresh)
    #cv2.imshow("Frame Delta", frameDelta)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break
    if key == ord('p'):
        time.sleep(5)
cv2.destroyAllWindows()
vs.release()

15.9 s ± 60.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
